# 🚀 Enhanced Breakout Scanner Dashboard - Top 3 Stocks Per Day

Interactive visualization dashboard for breakout scanner performance analysis with plotly grid layout.
Shows only the top 3 stocks per day with highest probability scores for focused trading opportunities.

In [1]:
# Setup and Imports
import sys
import os
from pathlib import Path
from datetime import datetime, date, time, timedelta
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

# Add project root to Python path
current_dir = Path.cwd()
if current_dir.name == 'scanner':
    project_root = current_dir.parent.parent
elif current_dir.name == 'notebook':
    project_root = current_dir.parent
else:
    project_root = current_dir

sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / 'src'))

# Import project modules
from src.infrastructure.logging import setup_logging
from src.application.scanners.strategies.breakout_scanner import BreakoutScanner
from src.infrastructure.core.database import DuckDBManager
from src.infrastructure.config.settings import get_settings

# Setup
setup_logging()

# Set up dark mode theme for plots
pio.templates.default = "plotly_dark"

# Custom dark theme configuration
dark_template = {
    "layout": {
        "paper_bgcolor": "#1e1e1e",
        "plot_bgcolor": "#1e1e1e",
        "font": {"color": "#ffffff"},
        "xaxis": {
            "gridcolor": "#333333",
            "linecolor": "#666666",
            "tickcolor": "#666666",
            "tickfont": {"color": "#ffffff"}
        },
        "yaxis": {
            "gridcolor": "#333333",
            "linecolor": "#666666",
            "tickcolor": "#666666",
            "tickfont": {"color": "#ffffff"}
        }
    }
}

print("✅ Dashboard setup complete with dark mode!")

✅ Dashboard setup complete with dark mode!


In [2]:
# Initialize Scanner
print("🔧 Initializing Breakout Scanner...")

# Initialize database and scanner
main_db_path = project_root / "data" / "financial_data.duckdb"
print(f"Database path: {main_db_path}")
print(f"Database exists: {main_db_path.exists()}")

db_manager = DuckDBManager(db_path=str(main_db_path))
scanner = BreakoutScanner(db_manager=db_manager)

# Test database connection
try:
    connection = db_manager.connect()
    print(f"✅ Database connection established: {connection is not None}")
    print(f"Connection type: {type(connection)}")
except Exception as e:
    print(f"❌ Database connection failed: {e}")
    import traceback
    traceback.print_exc()

print("✅ Scanner initialized!")

INFO:src.infrastructure.core.database:Initialized DuckDB manager with database: /Users/apple/Downloads/duckDbData/data/financial_data.duckdb
INFO:src.infrastructure.core.database:Connected to DuckDB database: /Users/apple/Downloads/duckDbData/data/financial_data.duckdb


🔧 Initializing Breakout Scanner...
Database path: /Users/apple/Downloads/duckDbData/data/financial_data.duckdb
Database exists: True
✅ Database connection established: True
Connection type: <class 'duckdb.duckdb.DuckDBPyConnection'>
✅ Scanner initialized!


In [3]:
# Run Enhanced Analysis
print("🆕 Running Enhanced Date Range Analysis...")

# Configure analysis parameters
start_date = date(2025, 9, 5)
end_date = date(2025, 9, 5)
breakout_time = time(9, 50)
end_of_day_time = time(15, 15)

print(f"📅 Analyzing period: {start_date} to {end_date}")
print(f"⏰ Breakout detection: {breakout_time}")
print(f"⏰ End-of-day analysis: {end_of_day_time}")

# Test database connection before running analysis
try:
    test_connection = db_manager.connect()
    print(f"🔗 Database connection test: {test_connection is not None}")
except Exception as e:
    print(f"❌ Database connection test failed: {e}")

# Run enhanced analysis
try:
    enhanced_results = scanner.scan_date_range(
        start_date=start_date,
        end_date=end_date,
        cutoff_time=breakout_time,
        end_of_day_time=end_of_day_time
    )
    
    if enhanced_results:
        print(f"\n✅ Found {len(enhanced_results)} enhanced breakout results")
        
        # Show sample results
        if len(enhanced_results) > 0:
            sample = enhanced_results[0]
            print(f"📊 Sample result keys: {list(sample.keys())}")
        
        # Export to CSV
        csv_filename = f"enhanced_breakout_analysis_{start_date.strftime('%Y%m%d')}_to_{end_date.strftime('%Y%m%d')}.csv"
        scanner.export_results(enhanced_results, csv_filename)
        print(f"💾 Results exported to: {csv_filename}")
    else:
        print("⚠️ No enhanced results found")
        
except Exception as e:
    print(f"❌ Enhanced analysis failed: {e}")
    import traceback
    traceback.print_exc()

print("\n🎉 Enhanced analysis complete!")

🆕 Running Enhanced Date Range Analysis...
📅 Analyzing period: 2025-09-05 to 2025-09-05
⏰ Breakout detection: 09:50:00
⏰ End-of-day analysis: 15:15:00
🔗 Database connection test: True
🚀 Enhanced Breakout Scanner - Date Range: 2025-09-05 to 2025-09-05
⏰ Breakout Detection: 09:50:00, End of Day: 15:15:00
🎯 Selecting Top 3 Stocks Per Day with Highest Probability
📅 Found 1 trading days to analyze
📊 Scanning 2025-09-05...
   ✅ Found 3 breakout candidates

✅ Enhanced Breakout Analysis Complete!
📊 Total breakout opportunities: 3
📅 Trading days analyzed: 1

✅ Found 3 enhanced breakout results
📊 Sample result keys: ['symbol', 'breakout_price', 'open_price', 'current_high', 'current_low', 'current_volume', 'breakout_above_resistance', 'breakout_pct', 'volume_ratio', 'probability_score', 'scan_date', 'breakout_time', 'eod_price', 'eod_high', 'eod_low', 'eod_volume', 'price_change_pct', 'breakout_successful', 'day_range_pct', 'performance_rank', 'overall_success_rate']
💾 Results exported to enhance

# 📊 Performance Dashboard

Interactive 2x3 grid visualization of breakout scanner performance.

In [4]:
# Performance Dashboard with Plotly 2x3 Grid Layout
print("📊 Creating Performance Dashboard...")

# Check if we have enhanced results
if 'enhanced_results' not in locals() or not enhanced_results:
    print("❌ No enhanced results available. Please run the Enhanced Analysis cell first.")
else:
    print(f"📈 Analyzing {len(enhanced_results)} breakout opportunities...")
    
    # Convert to DataFrame for analysis
    df = pd.DataFrame(enhanced_results)
    
    # Create 2x3 subplot layout
    fig = make_subplots(
        rows=2, cols=3,
        subplot_titles=(
            'Performance Summary', 'Daily Performance', 'Success Distribution',
            'Top Performers', 'Price Change Distribution', 'Volume vs Price Change'
        ),
        specs=[
            [{'type': 'table'}, {'type': 'bar'}, {'type': 'pie'}],
            [{'type': 'table'}, {'type': 'histogram'}, {'type': 'scatter'}]
        ],
        vertical_spacing=0.12,
        horizontal_spacing=0.05
    )
    
    # ===== 1. PERFORMANCE SUMMARY TABLE =====
    total_breakouts = len(df)
    successful_breakouts = len(df[df['breakout_successful'] == True])
    success_rate = (successful_breakouts / total_breakouts * 100) if total_breakouts > 0 else 0
    
    summary_data = {
        'Metric': ['Total Breakouts', 'Successful Breakouts', 'Success Rate', 'Avg Price Change', 'High Performers'],
        'Value': [
            total_breakouts,
            successful_breakouts,
            f"{success_rate:.1f}%",
            f"{df['price_change_pct'].mean():.2f}%" if 'price_change_pct' in df.columns else 'N/A',
            len(df[df['price_change_pct'] > 2.0]) if 'price_change_pct' in df.columns else 0
        ]
    }
    
    fig.add_trace(
        go.Table(
            header=dict(values=['<b>Performance Metrics</b>', '<b>Values</b>'],
                      fill_color='#2E4F7F', align='left', font=dict(size=12, color='white')),
            cells=dict(values=[summary_data['Metric'], summary_data['Value']],
                     fill_color='#1e1e1e', align='left', font=dict(size=11, color='white'))
        ),
        row=1, col=1
    )
    
    # ===== 2. DAILY PERFORMANCE BAR CHART =====
    if 'scan_date' in df.columns:
        try:
            # Convert scan_date to datetime if needed
            if not pd.api.types.is_datetime64_any_dtype(df['scan_date']):
                df['scan_date'] = pd.to_datetime(df['scan_date'])
            
            daily_stats = df.groupby(df['scan_date'].dt.date).agg({
                'symbol': 'count',
                'breakout_successful': 'sum'
            }).reset_index()
            daily_stats.columns = ['date', 'total', 'successful']
            
            fig.add_trace(
                go.Bar(
                    x=daily_stats['date'],
                    y=daily_stats['successful'],
                    name='Successful',
                    marker_color='green',
                    text=daily_stats['successful'],
                    textposition='auto'
                ),
                row=1, col=2
            )
            
            fig.add_trace(
                go.Bar(
                    x=daily_stats['date'],
                    y=daily_stats['total'] - daily_stats['successful'],
                    name='Unsuccessful',
                    marker_color='red',
                    text=daily_stats['total'] - daily_stats['successful'],
                    textposition='auto'
                ),
                row=1, col=2
            )
            
        except Exception as e:
            print(f"Note: Could not create daily chart: {e}")
    
    # ===== 3. SUCCESS DISTRIBUTION PIE CHART =====
    success_counts = df['breakout_successful'].value_counts()
    labels = ['Successful' if idx else 'Unsuccessful' for idx in success_counts.index]
    
    fig.add_trace(
        go.Pie(
            labels=labels,
            values=success_counts.values,
            marker_colors=['green', 'red'],
            textinfo='label+percent',
            title='Breakout Success Rate'
        ),
        row=1, col=3
    )
    
    # ===== 4. TOP PERFORMERS TABLE =====
    if 'performance_rank' in df.columns:
        top_performers = df.nsmallest(5, 'performance_rank')
        table_data = []
        
        for _, row in top_performers.iterrows():
            symbol = row['symbol']
            change = f"{row['price_change_pct']:+.2f}%" if 'price_change_pct' in row.index else 'N/A'
            rank = f"{row['performance_rank']:.2f}" if 'performance_rank' in row.index else 'N/A'
            table_data.append([symbol, change, rank])
        
        fig.add_trace(
            go.Table(
                header=dict(values=['<b>Symbol</b>', '<b>Price Change</b>', '<b>Rank</b>'],
                          fill_color='#2E8B57', align='left', font=dict(size=11, color='white')),
                cells=dict(values=list(zip(*table_data)) if table_data else [['N/A'], ['N/A'], ['N/A']],
                         fill_color='#1e1e1e', align='left', font=dict(size=10, color='white'))
            ),
            row=2, col=1
        )
    
    # ===== 5. PRICE CHANGE DISTRIBUTION =====
    if 'price_change_pct' in df.columns:
        fig.add_trace(
            go.Histogram(
                x=df['price_change_pct'],
                nbinsx=20,
                marker_color='blue',
                opacity=0.7,
                name='Price Changes'
            ),
            row=2, col=2
        )
    
    # ===== 6. VOLUME VS PRICE CHANGE SCATTER =====
    if 'volume_ratio' in df.columns and 'price_change_pct' in df.columns:
        fig.add_trace(
            go.Scatter(
                x=df['volume_ratio'],
                y=df['price_change_pct'],
                mode='markers',
                marker=dict(
                    size=8,
                    color=df['breakout_successful'].map({True: 'green', False: 'red'}),
                    showscale=False
                ),
                name='Volume vs Price Change',
                text=df['symbol']
            ),
            row=2, col=3
        )
    
    # Update layout
    fig.update_layout(
        height=800,
        width=1200,
        title_text='<b>🚀 Breakout Scanner Performance Dashboard</b>',
        title_x=0.5,
        title_font=dict(size=20, color='white'),
        showlegend=True,
        template='plotly_dark',
        paper_bgcolor='#1e1e1e',
        plot_bgcolor='#1e1e1e'
    )
    
    # Update subplot titles and axes with dark mode styling
    fig.update_xaxes(title_text='Date', row=1, col=2, title_font=dict(color='white'))
    fig.update_yaxes(title_text='Count', row=1, col=2, title_font=dict(color='white'))
    fig.update_xaxes(title_text='Price Change %', row=2, col=2, title_font=dict(color='white'))
    fig.update_yaxes(title_text='Frequency', row=2, col=2, title_font=dict(color='white'))
    fig.update_xaxes(title_text='Volume Ratio', row=2, col=3, title_font=dict(color='white'))
    fig.update_yaxes(title_text='Price Change %', row=2, col=3, title_font=dict(color='white'))
    
    # Apply dark mode to all axes
    fig.update_xaxes(gridcolor='#333333', linecolor='#666666', tickfont=dict(color='white'))
    fig.update_yaxes(gridcolor='#333333', linecolor='#666666', tickfont=dict(color='white'))
    
    # Show the dashboard
    fig.show()
    
    print("\n✅ Performance dashboard created successfully!")
    print(f"📊 Analyzed {total_breakouts} breakout opportunities")
    print(f"🎯 Success rate: {success_rate:.1f}%")

print("\n🎉 Performance Dashboard Complete!")

📊 Creating Performance Dashboard...
📈 Analyzing 3 breakout opportunities...



✅ Performance dashboard created successfully!
📊 Analyzed 3 breakout opportunities
🎯 Success rate: 66.7%

🎉 Performance Dashboard Complete!


# 📈 Dashboard Features

This dashboard provides:
- **Performance Summary**: Key metrics in table format
- **Daily Performance**: Success/failure by date
- **Success Distribution**: Pie chart of breakout outcomes
- **Top Performers**: Best performing stocks
- **Price Change Distribution**: Histogram of price movements
- **Volume vs Price Change**: Scatter plot showing volume-price relationship

All visualizations are interactive and can be zoomed, hovered, and filtered.